In [77]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

In [78]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [79]:
warnings.filterwarnings('ignore')

In [80]:
path_mapping = "/content/drive/MyDrive/ALU/datasets/id_mapping.csv"
path_social_profiles = "/content/drive/MyDrive/ALU/datasets/customer_social_profiles.csv"
path_transactions_aug = "/content/drive/MyDrive/ALU/datasets/customer_transactions_augmented.csv"

mappings = pd.read_csv(path_mapping)
profiles = pd.read_csv(path_social_profiles)
transactions = pd.read_csv(path_transactions_aug)

In [81]:
print("mapping shape")
print(mappings.shape)
print("social profiles shape")
print(profiles.shape)
print("transactions shape")
print(transactions.shape)

mapping shape
(155, 2)
social profiles shape
(155, 5)
transactions shape
(250, 6)


Checking for duplicates

In [82]:

duplicate_legacy_ids = mappings[mappings.duplicated(subset=['customer_id_legacy'], keep=False)]
duplicate_new_ids = mappings[mappings.duplicated(subset=['customer_id_new'], keep=False)]

print("Duplicate legacy IDs:")
print(duplicate_legacy_ids.shape[0])
print("\nDuplicate new IDs:")
print(duplicate_new_ids.shape[0])

if not duplicate_legacy_ids.empty:
    print("\nSample duplicate legacy IDs:")
    display(duplicate_legacy_ids.head())

Duplicate legacy IDs:
125

Duplicate new IDs:
129

Sample duplicate legacy IDs:


,customer_id_legacy,customer_id_new
0,195,A105
2,192,A156
3,157,A168
4,166,A102
5,155,A100


Merge Transactions with ID mapping

In [83]:
clean_mappings = mappings.drop_duplicates(subset=['customer_id_legacy'], keep='first')

merge_1 = pd.merge(transactions, clean_mappings, on="customer_id_legacy", how="left")

Merge with social profile using new customer IDS

In [84]:
# Merge with social profile using new customer IDs
merge_2 = pd.merge(merge_1, profiles, on="customer_id_new", how="left")

In [85]:
print(f"After second merge: {merge_2.shape}")
display(merge_2.head())

After second merge: (396, 11)


,customer_id_legacy,transaction_id,purchase_amount,purchase_date,product_category,customer_rating,customer_id_new,social_media_platform,engagement_score,purchase_interest_score,review_sentiment
0,151,1001,408.0,2024-01-01,Sports,2.3,A176,Facebook,70.0,3.3,Negative
1,192,1002,332.0,2024-01-02,Electronics,4.2,A156,Instagram,52.0,4.9,Positive
2,114,1003,442.0,2024-01-03,Electronics,2.1,A171,NaN,NaN,NaN,NaN
3,171,1004,256.0,2024-01-04,Clothing,2.8,NaN,NaN,NaN,NaN,NaN
4,160,1005,64.0,2024-01-05,Clothing,1.3,NaN,NaN,NaN,NaN,NaN


Handling Duplicates

In [86]:
merge_2['purchase_date'] = pd.to_datetime(merge_2['purchase_date'])

reference_date = merge_2['purchase_date'].max() + pd.Timedelta(days=1)

# Calculate days since last purchase
merge_2['days_since_purchase'] = (reference_date - merge_2['purchase_date']).dt.days

In [92]:
customer_aggs = merge_2.groupby('customer_id_legacy').agg(
    total_spent=('purchase_amount', 'sum'),
    avg_purchase=('purchase_amount', 'mean'),
    purchase_count=('transaction_id', 'count'),
    avg_rating=('customer_rating', 'mean'),
    first_purchase_date=('purchase_date', 'min'),  # Renamed to avoid confusion
    last_purchase_date=('purchase_date', 'max'),  # Renamed to avoid confusion
    avg_engagement=('engagement_score', 'mean'),
    avg_interest=('purchase_interest_score', 'mean')
).reset_index()

# Calculate days between first and last purchase (customer tenure)
customer_aggs['customer_tenure'] = (customer_aggs['last_purchase_date'] - customer_aggs['first_purchase_date']).dt.days
customer_aggs['customer_tenure'] = customer_aggs['customer_tenure'].fillna(0)

# Calculate purchase frequency (purchases per day of tenure)
customer_aggs['purchase_frequency'] = customer_aggs.apply(
    lambda x: x['purchase_count'] / (x['customer_tenure'] + 1), axis=1)

# Calculate monetary value (average purchase amount)
customer_aggs['monetary_value'] = customer_aggs['avg_purchase']

# Create customer engagement score
customer_aggs['engagement_score'] = (
    0.3 * customer_aggs['avg_engagement'].fillna(customer_aggs['avg_engagement'].mean()) / 100 +
    0.3 * customer_aggs['purchase_frequency'] / customer_aggs['purchase_frequency'].max() +
    0.4 * customer_aggs['monetary_value'] / customer_aggs['monetary_value'].max()
)

# Scale the engagement score from 0-100
customer_aggs['engagement_score'] = customer_aggs['engagement_score'] * 100
customer_aggs['engagement_score'] = customer_aggs['engagement_score'].clip(0, 100)

In [88]:
# Sort data by customer and purchase date
merge_2_sorted = merge_2.sort_values(['customer_id_legacy', 'purchase_date'])

# Calculate rolling average of last 3 purchases (if available)
merge_2_sorted['rolling_avg_3'] = merge_2_sorted.groupby('customer_id_legacy')['purchase_amount'].transform(
    lambda x: x.rolling(window=3, min_periods=1).mean())

# Calculate month of purchase for time-based aggregation
merge_2_sorted['purchase_month'] = merge_2_sorted['purchase_date'].dt.to_period('M')

In [89]:
def create_sentiment_features(df):
    # Filter for non-null sentiment
    sentiment_map = {'Positive': 1, 'Negative': -1, 'Neutral': 0}
    df['sentiment_score'] = df['review_sentiment'].map(sentiment_map).fillna(0)
    return df

# Apply sentiment feature creation
merge_2_processed = create_sentiment_features(merge_2_sorted)

# Calculate reference date for recency calculation
reference_date = merge_2['purchase_date'].max() + pd.Timedelta(days=1)

# Calculate days since last purchase (recency)
merge_2_processed['days_since_purchase'] = (reference_date - merge_2_processed['purchase_date']).dt.days

In [93]:
final_dataset = pd.merge(merge_2_processed,
                         customer_aggs[['customer_id_legacy', 'engagement_score',
                                        'purchase_frequency', 'customer_tenure',
                                        'first_purchase_date']],
                         on='customer_id_legacy',
                         how='left',
                         suffixes=('', '_agg'))

final_dataset['days_since_first_purchase'] = (reference_date - final_dataset['first_purchase_date']).dt.days

# Create category
category_pivot = pd.pivot_table(merge_2,
                               values='purchase_amount',
                               index='customer_id_legacy',
                               columns='product_category',
                               aggfunc='sum').fillna(0)

In [94]:
category_totals = category_pivot.sum(axis=1)
for column in category_pivot.columns:
    category_pivot[f'{column}_pct'] = category_pivot[column] / category_totals * 100

# Merge category preferences with main dataset
final_dataset = pd.merge(final_dataset,
                         category_pivot.reset_index(),
                         on='customer_id_legacy',
                         how='left')

# Select relevant columns
final_columns = [
    'customer_id_legacy', 'customer_id_new', 'purchase_amount', 'purchase_date',
    'product_category', 'customer_rating', 'social_media_platform', 'engagement_score',
    'purchase_interest_score', 'review_sentiment', 'sentiment_score', 'days_since_purchase',
    'rolling_avg_3', 'purchase_frequency', 'customer_tenure', 'days_since_first_purchase'
]

category_columns = [col for col in final_dataset.columns if '_pct' in col]
final_columns.extend(category_columns)

In [95]:
export_columns = [col for col in final_columns if col in final_dataset.columns]
export_dataset = final_dataset[export_columns].copy()

# Export the dataset
export_dataset.to_csv('final_customer_data_group1.csv', index=False)
print("Final dataset exported successfully with shape:", export_dataset.shape)

Final dataset exported successfully with shape: (396, 20)
